In [14]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
print("Workspace: " + ws.name, "Region: " + ws.location, sep = '\n')
#ws = Workspace.get(name="AzureML", resource_group_name="Corso-MS-Cloud")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace: azureml
Region: westeurope
Workspace name: azureml
Azure region: westeurope
Subscription id: 5d71b05c-3bd6-4055-836d-99b7558bd920
Resource group: corso-ms-cloud


In [15]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
aml_compute_cluster_name = 'cpu-cluster'
try:
    aml_compute = ComputeTarget(workspace=ws, name=aml_compute_cluster_name)
    print('Found cluster name, using it...')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_siza='Standard_D2_V2',
                                                            max_nodes=4)
    aml_compute = ComputeTarget.create(ws, aml_compute_cluster_name, compute_config)
aml_compute.wait_for_completion(show_output=True)

Found cluster name, using it...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [22]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
        "--max_iter" : choice(5,10,20,40),
        "--C": uniform(0.5,1.0)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 2, slack_factor = 0.1,delay_evaluation = 1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./",entry_script = 'train.py',compute_target = aml_compute)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                    hyperparameter_sampling = ps,
                                    policy = policy,
                                    primary_metric_name = 'Accuracy',
                                    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs = 25,
                                    max_concurrent_runs = 4)

In [23]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run = exp.submit(config = hyperdrive_config)
### YOUR CODE HERE ###
# submit script to AML
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [28]:
assert(run.get_status() == "Completed")

In [29]:
import joblib
# Get your best run and save the model from that run.
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
### YOUR CODE HERE ###

Best Run Id:  HD_6f4dea8e-957a-43db-9ef8-b0a69ae49e00_5

 Accuracy: 0.9146637533381888


In [30]:
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '0.9119370655785697', '--max_iter', '10']


In [31]:
model = best_run.register_model(model_name = 'best_model', model_path = './')

In [32]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
 
### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path= url)

In [33]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

TabularDataset
{
  "source": [
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}


In [34]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=10,
    compute_target= aml_compute,
    iterations=5)

In [35]:
# Submit your automl run

### YOUR CODE HERE ###
experiment = Experiment(ws, "automl_exp")
run = experiment.submit(config = automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
automl_exp,AutoML_c40e8d82-9e25-4b09-b352-1925e7030687,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

In [36]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, best_model = run.get_output()
best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')

Model(workspace=Workspace.create(name='azureml', subscription_id='5d71b05c-3bd6-4055-836d-99b7558bd920', resource_group='corso-ms-cloud'), name=automl_best_model.pkl, id=automl_best_model.pkl:1, version=1, tags={}, properties={})

In [ ]:
aml_compute.delete()